In [ ]:
import os
import sys

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sys.path += ['/content/drive/Shareddrive/filtered_data']

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%cd /content/drive/Shareddrives/cs145project/filtered_data/stocks
%cd ../..
%cd headlines/sentiment_analysis_financial_news

/content/drive/Shareddrives/cs145project/filtered_data/stocks
/content/drive/Shareddrives/cs145project
/content/drive/Shareddrives/cs145project/headlines/sentiment_analysis_financial_news


In [ ]:
path="all-data.csv"
sentiments=pd.read_csv(path,encoding = "ISO-8859-1")

In [ ]:
# Change Directories Again
%cd /content/drive/Shareddrives/cs145project/filtered_data/

path="top-50-analyst-ratings-processed.csv"
headlines=pd.read_csv(path,encoding = "ISO-8859-1")

/content/drive/Shareddrives/cs145project/filtered_data


In [ ]:
print(f"Cardinatlity of sentiments {len(sentiments[sentiments.columns[1]])}")
print(f"Cardinatlity of sentiments set {len(set(sentiments[sentiments.columns[1]]))}")

print(f"Cardinality of headlines {len(headlines['title'])}")
print(f"Cardinality of headlines set {len(set(headlines['title']))}")

diff = set(sentiments[sentiments.columns[1]]) - set(headlines['title'])
print (f"Cardinality of (sentiments - headlines) set difference {len(diff)}")

Cardinatlity of sentiments 4845
Cardinatlity of sentiments set 4837
Cardinality of headlines 116993
Cardinality of headlines set 93149
Cardinality of (sentiments - headlines) set difference 4837


In [ ]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

from sklearn.preprocessing import LabelEncoder

class myEnsembleVoteClassifier:
  def __init__(self, clfs, *args, **kwargs):
    self.clfs = clfs
    self.le_ = None
    
  def predict(self, X):
    predictions = self._predict(X)

    maj = np.apply_along_axis(
        lambda x: np.argmax(np.bincount(x)),
        axis=1, arr=predictions)
    return self.le_.inverse_transform(maj)
  
  def _predict(self, X):
    predictions = [clf.predict(X) for clf in self.clfs]
    if not self.le_:
      print('setting le_')
      self.le_ = LabelEncoder().fit(predictions[0])
      
    return np.asarray(
        [self.le_.transform(p) for p in predictions]).T

def get_predictions(vector):
  model_names = ['sgd', 'spacy', 'bert']
  models = []
  for name in model_names:
    try:
      # NOTE: this relies on the unpickle_obj as defined in the main notebook
      models.append(unpickle_obj(name))
    except:
      print(f"No pickle for '{name}' found")
  
  ensemble = myEnsembleVoteClassifier(models)
  prediction = ensemble.predict(vector)

  return prediction


In [ ]:
import numpy as np
import pandas as pd
import os
import sys

from google.colab import drive
drive.mount('/content/drive')
sys.path += ['/content/drive/Shareddrive/filtered_data']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/cs145project/filtered_data


In [ ]:
%cd /content/drive/Shareddrives/cs145project/filtered_data/
majority_pickle_path = '/content/drive/Shareddrives/cs145project/filtered_data/majority_vote_sentiment.pkl'

majority_df = pd.read_pickle(majority_pickle_path)

/content/drive/Shareddrives/cs145project/filtered_data


In [ ]:
news_path = '/content/drive/Shareddrives/cs145project/split_data/train_news/ADBE-analyst-rating-train.csv'

news = pd.read_csv(news_path)

def get_pred(df):
  return majority_df.loc[majority_df['title'].isin(df['title']) & majority_df['stock'].isin(df['stock'])]

prd = get_pred(news)
print(majority_df.describe())
print(news.describe())
print(prd.describe())

                          title                       date   stock sentiment
count                    116993                     116993  116993    116993
unique                    93149                      95163      50         3
top     Benzinga's Top Upgrades  2020-03-12 12:10:00-04:00     MRK   neutral
freq                        374                         27    3232     90022
                             title        date stock
count                         1960        1960  1960
unique                        1841         749     1
top     Benzinga's Top Initiations  2010-09-22  ADBE
freq                            11          26  1960
                             title                       date stock sentiment
count                         1939                       1939  1939      1939
unique                        1819                       1897     1         3
top     Benzinga's Top Initiations  2012-06-19 16:08:00-04:00  ADBE   neutral
freq                            11    